In [1]:
!pip install psycopg2 sqlalchemy python-dotenv pandas

In [2]:
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

In [3]:
#load enviornment variables from .env file
load_dotenv()

db_user  = os.environ.get('POSTGRES_USER')
db_password  = os.environ.get('POSTGRES_PASSWORD')
db_name  = os.environ.get('POSTGRES_DB')
db_host  = 'localhost'
db_port  = '5432'


In [4]:
# use psycopg to make connection
try:
    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        dbname=db_name,
        user=db_user,
        password=db_password
    )
    print("Database Postgres connection successful")

except psycopg2.Error as e:
    print("Error connecting to the database",e)

Database Postgres connection successful


In [5]:
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_user}')

In [6]:
csv_files_directory = "./data"

if not os.path.exists(csv_files_directory):
    print(f'Directory {csv_files_directory} does not exist')
else:
    csv_files = [f for f in os.listdir(csv_files_directory) if f.endswith('.csv')]

In [7]:
csv_files

['CUST_AZ12.csv',
 'cust_info.csv',
 'LOC_A101.csv',
 'prd_info.csv',
 'PX_CAT_G1V2.csv',
 'sales_details.csv']

In [1]:
import pandas as pd
import os

# Path where your CSVs are
csv_folder = './data'

# List of CSV files
csv_files = [
    "CUST_AZ12.csv",
    "cust_info.csv",
    "LOC_A101.csv",
    "prd_info.csv",
    "PX_CAT_G1V2.csv",
    "sales_details.csv"
    
]

for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        
        # Drop duplicates
        df_cleaned = df.drop_duplicates()
        
        # Overwrite the original file
        df_cleaned.to_csv(file_path, index=False)
        
        print(f"✅ Cleaned and overwritten: {file_path}")
    else:
        print(f"❌ File not found: {file_path}")

print("🚀 All CSVs cleaned and updated successfully!")


✅ Cleaned and overwritten: ./data\CUST_AZ12.csv
✅ Cleaned and overwritten: ./data\cust_info.csv
✅ Cleaned and overwritten: ./data\LOC_A101.csv
✅ Cleaned and overwritten: ./data\prd_info.csv
✅ Cleaned and overwritten: ./data\PX_CAT_G1V2.csv
✅ Cleaned and overwritten: ./data\sales_details.csv
🚀 All CSVs cleaned and updated successfully!


In [13]:
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        # Check nulls
        null_counts = df.isnull().sum()

        print(f"\n📄 {file}")
        print(null_counts)
    else:
        print(f"❌ File not found: {file_path}")


📄 CUST_AZ12.csv
CID         0
BDATE       0
GEN      1472
dtype: int64

📄 cust_info.csv
cst_id                   4
cst_key                  0
cst_firstname            8
cst_lastname             7
cst_marital_status       7
cst_gndr              4578
cst_create_date          4
dtype: int64

📄 LOC_A101.csv
CID        0
CNTRY    332
dtype: int64

📄 prd_info.csv
prd_id            0
prd_key           0
prd_nm            0
prd_cost          2
prd_line         17
prd_start_dt      0
prd_end_dt      197
dtype: int64

📄 PX_CAT_G1V2.csv
ID             0
CAT            0
SUBCAT         0
MAINTENANCE    0
dtype: int64

📄 sales_details.csv
sls_ord_num     0
sls_prd_key     0
sls_cust_id     0
sls_order_dt    0
sls_ship_dt     0
sls_due_dt      0
sls_sales       8
sls_quantity    0
sls_price       7
dtype: int64


In [2]:
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Drop rows with any null values
        df = df.dropna()

        # Save it back
        df.to_csv(file_path, index=False)

        print(f"✅ Dropped nulls from {file}")
    else:
        print(f"❌ File not found: {file_path}")

print("\n✅ All files cleaned for nulls only.")

✅ Dropped nulls from CUST_AZ12.csv
✅ Dropped nulls from cust_info.csv
✅ Dropped nulls from LOC_A101.csv
✅ Dropped nulls from prd_info.csv
✅ Dropped nulls from PX_CAT_G1V2.csv
✅ Dropped nulls from sales_details.csv

✅ All files cleaned for nulls only.


In [3]:
import os
import pandas as pd

# Define correctly
csv_files_directory = "./data"  # <- folder where CSVs are stored
csv_files = [
    "CUST_AZ12.csv",
    "cust_info.csv",
    "LOC_A101.csv",
    "prd_info.csv",
    "PX_CAT_G1V2.csv",
    "sales_details.csv"
]

# Your PostgreSQL connection
from sqlalchemy import create_engine
import psycopg2

db_user = 'postgres'
db_password = 'example'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

connection = psycopg2.connect(
    database=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


def load_csv_to_postgresql(csv_file, table_name, conn, engine):
    try:
        df = pd.read_csv(csv_file)

        # Dynamically create the table
        columns = ', '.join([f'"{col}" TEXT' for col in df.columns])

        create_table_query = f"""
            CREATE TABLE IF NOT EXISTS "{table_name}" (
                {columns}
            );
        """

        print(f"Creating table '{table_name}' with query: {create_table_query}")

        with conn.cursor() as cursor:
            cursor.execute(create_table_query)
            conn.commit()

        # Load Data
        df.to_sql(table_name, engine, if_exists="replace", index=False, method='multi', chunksize=1000)
        print(f"✅ Data from {csv_file} loaded into '{table_name}' successfully.")

    except Exception as e:
        print(f"❌ Error loading {csv_file}: {e}")


# Start uploading
if not os.path.exists(csv_files_directory):
    print(f"❌ Directory {csv_files_directory} does not exist.")
else:
    for csv_file in csv_files:
        csv_file_path = os.path.join(csv_files_directory, csv_file)
        table_name = os.path.splitext(csv_file)[0].lower()  # table names lowercase
        load_csv_to_postgresql(csv_file=csv_file_path, table_name=table_name, conn=connection, engine=engine)

if connection:
    connection.close()
    print("🔒 Database connection closed.")


Creating table 'cust_az12' with query: 
            CREATE TABLE IF NOT EXISTS "cust_az12" (
                "CID" TEXT, "BDATE" TEXT, "GEN" TEXT
            );
        
✅ Data from ./data\CUST_AZ12.csv loaded into 'cust_az12' successfully.
Creating table 'cust_info' with query: 
            CREATE TABLE IF NOT EXISTS "cust_info" (
                "cst_id" TEXT, "cst_key" TEXT, "cst_firstname" TEXT, "cst_lastname" TEXT, "cst_marital_status" TEXT, "cst_gndr" TEXT, "cst_create_date" TEXT
            );
        
✅ Data from ./data\cust_info.csv loaded into 'cust_info' successfully.
Creating table 'loc_a101' with query: 
            CREATE TABLE IF NOT EXISTS "loc_a101" (
                "CID" TEXT, "CNTRY" TEXT
            );
        
✅ Data from ./data\LOC_A101.csv loaded into 'loc_a101' successfully.
Creating table 'prd_info' with query: 
            CREATE TABLE IF NOT EXISTS "prd_info" (
                "prd_id" TEXT, "prd_key" TEXT, "prd_nm" TEXT, "prd_cost" TEXT, "prd_line" TEXT, "p